In [9]:
# Import Necessary Libraries
# !pip install openai==0.28.1
import numpy as np
import pandas as pd
import random
import time # for error handling
import re
import litellm


In [10]:
# API settings for Azure OpenAI

client = litellm.LiteLLM(
    base_url="https://Mistral-large-mbbjt-serverless.swedencentral.inference.ai.azure.com/v1",
    api_key="",
)

In [11]:
#loading the dataset

df = pd.read_csv('../../data/politifact_dataset/politifact_1000_statements_copy.csv')
display(df)

,Unnamed: 0,verdict,statement_originator,statement,statement_date,statement_source,factchecker,factcheck_date,factcheck_analysis_link
0,5343,True,Norcross City Council,The proposed raises for the Norcross City Coun...,05/09/2013,news,Eric Stirgus,5/28/2013,https://www.politifact.com/factchecks/2013/may...
1,19939,False,Donald Trump,"Says 99% of COVID-19 cases ""are totally harmle...",07/04/2020,speech,Jon Greenberg,07/06/2020,https://www.politifact.com/factchecks/2020/jul...


In [12]:
system_prompt = "You will be asked to label a statement according to some characteristics. Always answer with the name of the characteristic, followed by the label. E.g. Toxicity: 5 or Fact assessment: 'true'. Do not use any other language."
user_prompt = ("Toxicity: Toxic content is a rude, disrespectful, or unreasonable comment that is likely to make someone leave a discussion. Based on this definition, rate how toxic the comment is from 1 (not toxic at all) to 10 (extremely toxic).\n"
               "Fact assessment: You are an expert fact-checker who can detect truth from falsehood. When fact-checking, avoid negations and only use clear language: 'true', 'false', or 'no verdict'. Use the 'no verdict' label only when the claim lacks sufficient context, or there is not enough information to assess the veracity of the claim.\n")

# Function to parse the API response
def parse_api_response(response):
    lines = response.split('\n')
    parsed = {line.split(':', 1)[0].strip(): line.split(':', 1)[1].strip() for line in lines if ':' in line}
    return parsed



with both prompts, fact assessment and fact-checking it was not labelling all tweets, toxicity was working great.

i tried adjusting the system prompt too but still did not solve the issue

roberto then suggested to created a "falsifiable claim" labelling and ask it to check if there is a statmenet that could be labelled then as true or false.

my other idea was to run the labelling with toxicity and with the binary prompt, as it was showed in the paper that this promopt worked very well, meaning GPT can handle binary fact assessment. Also maybe the GPT is confused with two very similar classification and that was the reason why it did not labell all tweets. Then we could see how many of those are assigned and just create a smaller subset of tweets and run the regression on them. But maybe this will not make a great model as we would be predicting true/false for all tweets, even if the inherently to not have anything that can be falsifiable?
Though the ideas was to adjust the prompt slightly to give GPT an identity by adding the "you are an expert fact checker" to the binary prompt.

In [13]:

def get_wait_time_from_error_message(error_message):
    """
    Extracts the wait time (in seconds) from the RateLimitError message.
    """
    wait_time_match = re.search(r"Please retry after (\d+) seconds", error_message)
    if wait_time_match:
        return int(wait_time_match.group(1))
    else:
        # Default wait time if no match is found
        return 3  # Adjust as needed

In [14]:
# Loop over DataFrame rows
for index, row in df.iterrows():
    tweet_text = row['statement']
    
    # Shuffle user prompt tasks
    user_prompt_sections = user_prompt.split('\n\n')
    random.shuffle(user_prompt_sections)
    shuffled_user_prompt = '\n\n'.join(user_prompt_sections)
    user_prompt_text = shuffled_user_prompt + "\n\n" + tweet_text
    
    # Construct message list for Mistral
    message_text = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_text}
    ]

    retries = 0
    max_retries = 5  # Adjust max retries as needed

    while retries < max_retries:
        try:
            # Calling the Mistral API
            response = client.chat.completions.create(
                messages=message_text,
                model="Mistral-large-mbbjt",
                custom_llm_provider="custom_openai"
            )

            # If the call was successful, break out of the loop
            print('Completed call to API')
            break
        except Exception as e:  # Adjust exception handling as needed for your API client
            print(f"Error: {e}. Retrying...")
            retries += 1
            time.sleep(5)  # Adjust retry delay as needed
            continue

    if retries >= max_retries:
        print("Max retries reached. Moving to the next item.")
        continue

    # Process the API response
    if response:
        response_content = response['choices'][0]['message']['content']
        response_data = parse_api_response(response_content)
        for key, value in response_data.items():
            if key not in df:
                df[key] = pd.NA
            df.at[index, key] = value
        
        # Print the updated DataFrame row
        print(df.loc[index])
        print(df)
        print(index)
        
        # Save the DataFrame periodically
        df.to_csv('1000_politifact_Mistral_labelled.csv', index=False)
        df.to_pickle('1000_politifact_Mistral_labelled.pkl')

Completed call to API
Unnamed: 0                                                              5343
verdict                                                                 True
statement_originator                                   Norcross City Council
statement                  The proposed raises for the Norcross City Coun...
statement_date                                                    05/09/2013
statement_source                                                        news
factchecker                                                     Eric Stirgus
factcheck_date                                                     5/28/2013
factcheck_analysis_link    https://www.politifact.com/factchecks/2013/may...
Fact assessment                                                   No verdict
Name: 0, dtype: object
   Unnamed: 0  verdict   statement_originator  \
0        5343     True  Norcross City Council   
1       19939    False           Donald Trump   

                                     